In [255]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from joblib import dump, load
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

import joblib
import random

In [256]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [257]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


setup_seed(666)

In [258]:
new_file = 'out_validation.xlsx'

In [259]:
file = 'train_and_test.xlsx'

In [260]:
new_dataframe = pd.read_excel(file, sheet_name=2)

In [261]:
new_dataframe1 = pd.read_excel(new_file, sheet_name=0)

In [262]:
new_dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   性别      52 non-null     int64
 1   年龄      52 non-null     int64
 2   吸烟      52 non-null     int64
 3   原发灶大小   52 non-null     int64
 4   骨转移     52 non-null     int64
 5   脑转移     52 non-null     int64
 6   肝转      52 non-null     int64
 7   胸膜转移    52 non-null     int64
 8   联合治疗    52 non-null     int64
 9   转移个数    52 non-null     int64
 10  位点      52 non-null     int64
 11  TP53    52 non-null     int64
 12  PIK3CA  52 non-null     int64
 13  RB1     52 non-null     int64
 14  TMB     52 non-null     int64
 15  END     52 non-null     int64
dtypes: int64(16)
memory usage: 6.6 KB


In [263]:
def recall(y_test, y_pred):
    zero_length = len([i for i in y_test if i == 0])
    print("zero_length", zero_length)
#     print(y_test)
#     print(y_pred)
    zero_true_count = 0
    for i in range(len(y_test)):
        if y_test[i] == 0 and y_pred[i] == 0:
            zero_true_count += 1
    return zero_true_count / zero_length

In [264]:
# Dataset2
X_second = new_dataframe1.iloc[:,range(0,15)]
y_second = new_dataframe1.iloc[:, -1]  # 第二个指标
X_second = np.asarray(X_second)
X_second = torch.Tensor(X_second)
y_second = torch.Tensor(y_second)

In [265]:
# Dataset2
X_second1 = new_dataframe1.iloc[:,range(0,15)]
y_second1 = new_dataframe1.iloc[:, -1]  # 第二个指标
X_second1 = np.asarray(X_second1)

In [266]:
y_second1 = np.array(y_second1)

In [267]:
X_train_second, X_test_second, y_train_second, y_test_second = train_test_split(X_second1, y_second1, test_size=0.20, random_state=100)
X_train_second = torch.Tensor(X_train_second)
X_test_second = torch.Tensor(X_test_second)
y_train_second = torch.Tensor(y_train_second)
y_test_second = torch.Tensor(y_test_second)

In [268]:
class Feedforward_second(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward_second, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.batchnorm = torch.nn.BatchNorm1d(self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size, bias=True)
            self.dropout1 = torch.nn.Dropout(p=0.05)
            self.relu1 = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 16, bias=True)
            self.dropout2 = torch.nn.Dropout(p=0.05)
            self.relu2 = torch.nn.ReLU()
            self.fc3 = torch.nn.Linear(16, 1, bias=True)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
#             batchnorm = self.batchnorm(hidden)
            hidden = self.relu(hidden)
            relu = self.relu1(hidden)
            output = self.fc2(relu)
            output = self.relu2(output)
            output = self.fc3(output)
            output = self.sigmoid(output)
            return output

In [269]:
model2 = Feedforward_second(X_train_second.shape[1],20)
criterion2 = torch.nn.BCELoss()
optimizer2 = torch.optim.AdamW(model2.parameters(), lr = 0.005, weight_decay= 5e-4)
scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer2, gamma=0.92)

In [270]:
import datetime
now = str(datetime.datetime.now().year) + "_" + str(datetime.datetime.now().month) + "_" + str(datetime.datetime.now().day) + "_" + \
str(datetime.datetime.now().hour) + "_" + str(datetime.datetime.now().minute) + "_" + str(datetime.datetime.now().second)
model2.train()
epoch = 500
loss_array = []
highest_accuracy = 0
for epoch in range(epoch):
    optimizer2.zero_grad()
    # Forward pass
    y_pred = model2(X_train_second)
    # Compute Loss
    loss = criterion2(y_pred.squeeze(), y_train_second)
    loss_array.append(float(loss.item()))
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer2.step()
    model2.eval()
    y_pred_second = model2(X_test_second)
    y_pred_second_int = []
    for item in y_pred_second:
        y_pred_second_int.append(round(float(item[0])))
#     fpr2, tpr2, thersholds2 = roc_curve(y_test_second, y_pred_second_int, pos_label=1)
    test_acc = np.sum(y_pred_second_int==np.array(y_test_second))/len(y_test_second)
    if test_acc > highest_accuracy:
        highest_accuracy = test_acc
        torch.save(model2, f"ann_models/model2_{test_acc}.pt")


#     scheduler2.step()

Epoch 0: train loss: 0.7061502933502197
Epoch 1: train loss: 0.699498176574707
Epoch 2: train loss: 0.6953036189079285
Epoch 3: train loss: 0.6927807927131653
Epoch 4: train loss: 0.6905691623687744
Epoch 5: train loss: 0.6888396739959717
Epoch 6: train loss: 0.6875101923942566
Epoch 7: train loss: 0.6863274574279785
Epoch 8: train loss: 0.6851624250411987
Epoch 9: train loss: 0.6838745474815369
Epoch 10: train loss: 0.6824457049369812
Epoch 11: train loss: 0.680958092212677
Epoch 12: train loss: 0.6793220043182373
Epoch 13: train loss: 0.6773959994316101
Epoch 14: train loss: 0.6752957105636597
Epoch 15: train loss: 0.6728329062461853
Epoch 16: train loss: 0.6702690720558167
Epoch 17: train loss: 0.6675310134887695
Epoch 18: train loss: 0.6643450260162354
Epoch 19: train loss: 0.6607840657234192
Epoch 20: train loss: 0.6566256880760193
Epoch 21: train loss: 0.6519157290458679
Epoch 22: train loss: 0.6465938687324524
Epoch 23: train loss: 0.641169548034668
Epoch 24: train loss: 0.63547

Epoch 278: train loss: 0.016149960458278656
Epoch 279: train loss: 0.015972083434462547
Epoch 280: train loss: 0.01582026667892933
Epoch 281: train loss: 0.01564384438097477
Epoch 282: train loss: 0.015462471172213554
Epoch 283: train loss: 0.015297477133572102
Epoch 284: train loss: 0.015128478407859802
Epoch 285: train loss: 0.014971606433391571
Epoch 286: train loss: 0.014803868718445301
Epoch 287: train loss: 0.014660348184406757
Epoch 288: train loss: 0.01451551727950573
Epoch 289: train loss: 0.01435779221355915
Epoch 290: train loss: 0.014206787571310997
Epoch 291: train loss: 0.014041617512702942
Epoch 292: train loss: 0.013919119723141193
Epoch 293: train loss: 0.013779324479401112
Epoch 294: train loss: 0.013619761914014816
Epoch 295: train loss: 0.013493304140865803
Epoch 296: train loss: 0.013348481617867947
Epoch 297: train loss: 0.01321993675082922
Epoch 298: train loss: 0.013104837387800217
Epoch 299: train loss: 0.012966846115887165
Epoch 300: train loss: 0.012835871428

Epoch 477: train loss: 0.003474744036793709
Epoch 478: train loss: 0.0034543885849416256
Epoch 479: train loss: 0.0034394424874335527
Epoch 480: train loss: 0.0034164898097515106
Epoch 481: train loss: 0.003398740431293845
Epoch 482: train loss: 0.0033786639105528593
Epoch 483: train loss: 0.003363225143402815
Epoch 484: train loss: 0.0033437979873269796
Epoch 485: train loss: 0.0033264472149312496
Epoch 486: train loss: 0.0033064819872379303
Epoch 487: train loss: 0.0032927049323916435
Epoch 488: train loss: 0.0032714104745537043
Epoch 489: train loss: 0.0032553577329963446
Epoch 490: train loss: 0.003235970623791218
Epoch 491: train loss: 0.0032210492063313723
Epoch 492: train loss: 0.003201075131073594
Epoch 493: train loss: 0.003184799337759614
Epoch 494: train loss: 0.003170676063746214
Epoch 495: train loss: 0.0031531504355371
Epoch 496: train loss: 0.0031348634511232376
Epoch 497: train loss: 0.003118664026260376
Epoch 498: train loss: 0.00310256564989686
Epoch 499: train loss: 

In [271]:
highest_accuracy

0.8181818181818182

In [272]:
model2 = torch.load(f"ann_models/model2_{highest_accuracy}.pt")
# model2 = torch.load(f"ann_models/model2_{highest_accuracy}.pt")
model2.eval()
y_pred_second = model2(X_test_second)
y_pred_second_int = []

for item in y_pred_second:
    y_pred_second_int.append(round(float(item[0])))

print("internal validation accuracy:", np.sum(y_pred_second_int==np.array(y_test_second))/len(y_test_second))

internal validation accuracy: 0.8181818181818182


In [273]:
Y_OUT = model2(X_second)
Y_OUTs = []

In [274]:
for item in Y_OUT:
    Y_OUTs.append(round(float(item[0])))

In [275]:
print("external validation accuracy:", np.sum(Y_OUTs==np.array(y_second))/len(y_second))

external validation accuracy: 0.8653846153846154


In [276]:
print("external validation recall:", recall(Y_OUTs, y_second))

zero_length 22
external validation recall: 0.8636363636363636


In [277]:
fpr2, tpr2, thersholds2 = roc_curve(y_second, Y_OUTs, pos_label=1)
roc_auc2 = auc(fpr2, tpr2)
print("external validation auc:", recall(Y_OUTs, y_second))

zero_length 22
external validation auc: 0.8636363636363636
